In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import socceraction

from statsbombpy import sb

In [2]:
X = pd.read_csv('data/X_1516_LaLiga.csv', index_col=0)
# pd.Series({i: X[i].dtype for i in X.columns}) # 检查数据格式，都正确
Y = pd.read_csv('data/Y_1516_LaLiga.csv', index_col=0)

X.shape, Y.shape

((1126800, 46), (1126800, 2))

### 重新设计样本特征集（xG化为01二元因变量，并加入权重）

#### 1 score

In [3]:
score_xG_index = Y[(Y['scores'] > 0) & (Y['scores'] < 1)].index
score_0_index = Y[Y['scores'] == 0].index
score_1_index = Y[Y['scores'] == 1].index
print(len(score_xG_index), len(score_0_index), len(score_1_index))

X_score = pd.concat([X.loc[score_0_index], X.loc[score_1_index],
                     X.loc[score_xG_index], X.loc[score_xG_index]], ignore_index=True)
Y_score = np.concatenate([np.zeros(len(score_0_index)), np.ones(len(score_1_index)),
                          np.zeros(len(score_xG_index)), np.ones(len(score_xG_index))])
weight_score = np.concatenate([np.ones(len(score_0_index) + len(score_1_index)),
                               1 - Y.loc[score_xG_index, 'scores'].values,
                               Y.loc[score_xG_index, 'scores'].values])

X_score.shape, Y_score.shape, weight_score.shape

73647 1052019 1134


((1200447, 46), (1200447,), (1200447,))

#### 2 concede

In [4]:
concede_xG_index = Y[(Y['concedes'] > 0) & (Y['concedes'] < 1)].index
concede_0_index = Y[Y['concedes'] == 0].index
concede_1_index = Y[Y['concedes'] == 1].index
print(len(concede_xG_index), len(concede_0_index), len(concede_1_index))

X_concede = pd.concat([X.loc[concede_0_index], X.loc[concede_1_index],
                       X.loc[concede_xG_index], X.loc[concede_xG_index]], ignore_index=True)
Y_concede = np.concatenate([np.zeros(len(concede_0_index)), np.ones(len(concede_1_index)),
                            np.zeros(len(concede_xG_index)), np.ones(len(concede_xG_index))])
weight_concede = np.concatenate([np.ones(len(concede_0_index) + len(concede_1_index)),
                                 1 - Y.loc[concede_xG_index, 'concedes'].values,
                                 Y.loc[concede_xG_index, 'concedes'].values])

X_concede.shape, Y_concede.shape, weight_concede.shape

12248 1114418 134


((1139048, 46), (1139048,), (1139048,))

In [5]:
Y_score = pd.Series(Y_score, name='scores')
Y_concede = pd.Series(Y_concede, name='concedes')
weight_score = pd.Series(weight_score, name='weight')
weight_concede = pd.Series(weight_concede, name='weight')

X_score.to_csv('data/X_LaLiga_score.csv')
Y_score.to_csv('data/Y_LaLiga_score.csv')
weight_score.to_csv('data/weight_LaLiga_score.csv')
X_concede.to_csv('data/X_LaLiga_concede.csv')
Y_concede.to_csv('data/Y_LaLiga_concede.csv')
weight_concede.to_csv('data/weight_LaLiga_concede.csv')